In [1]:
# https://towardsdatascience.com/pandas-groupby-a-simple-but-detailed-tutorial-314b8f37005d
# https://towardsdatascience.com/accessing-data-in-a-multiindex-dataframe-in-pandas-569e8767201d
# https://towardsdatascience.com/summarizing-data-with-pandas-crosstab-efc8b9abecf
# https://towardsdatascience.com/how-to-flatten-multiindex-columns-and-rows-in-pandas-f5406c50e569
# https://datascientyst.com/list-aggregation-functions-aggfunc-groupby-pandas/
# https://stackoverflow.com/questions/25929319/how-to-iterate-over-pandas-multiindex-dataframe-using-index
# https://stackoverflow.com/questions/24495695/pandas-get-unique-multiindex-level-values-by-label
# https://stackoverflow.com/questions/55706391/pandas-crosstab-on-multiple-columns-then-groupby

# https://matplotlib.org/stable/gallery/pyplots/pyplot_text.html#sphx-glr-gallery-pyplots-pyplot-text-py

In [2]:
import pandas as pd
import numpy as np
from myUtils import pickle_load, pickle_dump

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

# # filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
# filename_symbols = path_data_dump + 'my_symbols.csv'  # symbols text file

# _filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_perf_rank_dict =  'perf_rank_dict'  # store symbols from performance rank results
filename_pickled_r_all_ranks =  'r_all_ranks'  # list of top 100 most common symbols from performance rank results
filename_pickled_df_a = 'df_OHLCV_clean'  # df adjusted OHLCV, dropped symbols with no vol and close
filename_pickled_df_c = 'df_close_clean'  # df close, dropped symbols with no vol and close

verbose = False  # True prints more output

#################
# look_back_days = -250 * 60  # subset df iloc days
look_back_days = -250 * 6  # subset df iloc days, 6 years of data
#################

In [3]:
print(f"Full path to pickled df_symbols_close:  {path_data_dump}{filename_pickled_df_symbols_close}")
df_close = pickle_load(path_data_dump, filename_pickled_df_symbols_close, verbose=verbose)
print(f"Full path to pickled df_adjOHLCV:  {path_data_dump}{filename_pickled_df_adjOHLCV}")
df_adjOHLCV = pickle_load(path_data_dump, filename_pickled_df_adjOHLCV, verbose=verbose)

Full path to pickled df_symbols_close:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_symbols_close
Full path to pickled df_adjOHLCV:  C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_adjOHLCV


In [4]:
# https://stackoverflow.com/questions/63826291/pandas-series-find-column-by-value
df = df_adjOHLCV[look_back_days::]
df_v = df.xs('Volume', level=1, axis=1)  # select only Volume columns
rows, cols = np.where(df_v == 0)  # row index, column index where trading volumes are zero
idx_no_volume = list(set(cols))
idx_no_volume.sort()
symbols_no_volume = df_v.columns[idx_no_volume]
print(f'symbols with no volume:\n{symbols_no_volume}')

symbols with no volume:
Index(['AMCR', 'AVT', 'BIIB', 'BKR', 'CCI', 'CHD', 'CNC', 'CNXC', 'ESAB',
       'FCFS', 'GEN', 'IBKR', 'JJC', 'KDNY', 'NEX', 'NUVA', 'PDCE', 'PEAK',
       'PFG', 'SBNY', 'SCHN', 'SRPT', 'VC', 'VNT', 'WH', 'XEL'],
      dtype='object')


In [5]:
df_dif = df_v - df_v.shift(periods=1)
rows, cols = np.where(df_dif == 0)
idx_same_volume = list(set(cols))
idx_same_volume.sort()
idx_same_volume
symbols_same_volume = df_v.columns[idx_same_volume]
print(f'symbols with same volume:\n{symbols_same_volume}')

symbols with same volume:
Index(['ACM', 'ACN', 'ADP', 'AGCO', 'ALRM', 'AMCR', 'AMED', 'AMG', 'AMGN',
       'ANSS',
       ...
       'VRE', 'VRSK', 'VSAT', 'WERN', 'WEX', 'WSFS', 'WTS', 'WTW', 'XEL',
       'XENE'],
      dtype='object', length=232)


In [6]:
df_c = df.xs('Close', level=1, axis=1)  # select only Close columns
df_c = df_c.fillna(0).copy()  # convert NaNs to zeros
rows, cols = np.where(df_c == 0)  # row index, column index where trading volumes are zero
idx_no_close = list(set(cols))
idx_no_close.sort()
symbols_no_close = df_c.columns[idx_no_close]
print(f'symbols with NaN close:\n{symbols_no_close}')

symbols with NaN close:
Index(['ABNB', 'AIRC', 'AJRD', 'ALGM', 'BJ', 'BKI', 'BRBR', 'BTC-USD', 'CARR',
       'CDAY', 'CEG', 'CHK', 'CHX', 'CNXC', 'CR', 'CRWD', 'CTVA', 'DDOG',
       'DOW', 'DT', 'DTM', 'ESAB', 'ETH-USD', 'ETRN', 'FOX', 'FOXA', 'FYBR',
       'GEHC', 'GFS', 'GO', 'GXO', 'JJC', 'KD', 'KDNY', 'LCID', 'MMP', 'MP',
       'MRNA', 'NARI', 'NEX', 'NUVA', 'NVST', 'NVT', 'OGN', 'OTIS', 'PDCE',
       'PDD', 'PGNY', 'RIVN', 'SGOV', 'SHC', 'SITM', 'SWAV', 'UNVR', 'VICI',
       'VNT', 'VSCO', 'WH', 'WWE', 'YETI', 'ZM', 'ZS'],
      dtype='object')


In [7]:
symbols_bad_data = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
unique_symbols_bad_data = sorted(list(set(symbols_bad_data)))  # unique symbols
print(f'unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: {len(unique_symbols_bad_data)}')

unique symbols with bad data, e.g. no volume, same volume and $0 close, includes duplicate symbols: 296


In [8]:
# get symbols of past model picks
df_picks = pickle_load(path_data_dump, 'df_picks', verbose=verbose)
df_picks.head(1)

,date_end_df_train,max_days_lookbacks,days_lookbacks,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,2023-09-18,120,"[30, 60, 120]",[],[],[],[],[],[],['SHV'],['FTSM'],[],"['CELH', 'DELL', 'LLY']","['CCJ', 'FTI', 'VST']","['ANF', 'ARCH', 'CLS', 'CPRI', 'HEP', 'HRB', '...","['CLMT', 'HZNP', 'SKYW']",[]


In [9]:
# keep only columns with symbols of past picks
df_picks.drop(columns=["date_end_df_train", "max_days_lookbacks", "days_lookbacks"], inplace=True)
df_picks.head(1)

,sym_freq_15,sym_freq_14,sym_freq_13,sym_freq_12,sym_freq_11,sym_freq_10,sym_freq_9,sym_freq_8,sym_freq_7,sym_freq_6,sym_freq_5,sym_freq_4,sym_freq_3,sym_freq_2
0,[],[],[],[],[],[],['SHV'],['FTSM'],[],"['CELH', 'DELL', 'LLY']","['CCJ', 'FTI', 'VST']","['ANF', 'ARCH', 'CLS', 'CPRI', 'HEP', 'HRB', '...","['CLMT', 'HZNP', 'SKYW']",[]


In [10]:
df_picks.columns

Index(['sym_freq_15', 'sym_freq_14', 'sym_freq_13', 'sym_freq_12',
       'sym_freq_11', 'sym_freq_10', 'sym_freq_9', 'sym_freq_8', 'sym_freq_7',
       'sym_freq_6', 'sym_freq_5', 'sym_freq_4', 'sym_freq_3', 'sym_freq_2'],
      dtype='object')

In [11]:
import ast

In [12]:
list_of_lists = []

for col in df_picks.columns:
  # convert column values from string to list, e.g. '[]', '[]', '["A", "B", ..]' ... to [], [], ["A", "B", ..], ...
  l_series = df_picks[col].apply(ast.literal_eval)
  # list_of_lists = [l_item for l_item in l_series if l_item]  # this doesn't works  
  for l_item in l_series:
      if l_item:  # 
        list_of_lists.append(l_item)  

symbols_picks = [val for sublist in list_of_lists for val in sublist]
print(f'symbol count from model picks: {len(symbols_picks)}')
# list sorted unique symbols
unique_symbols_picks = sorted(list(set(symbols_picks)))
print(f'unique symbol count from model picks: {len(unique_symbols_picks)}')

symbol count from model picks: 4680
unique symbol count from model picks: 248


In [13]:
def find_common_items(list1, list2):
  """Finds the common items between two lists.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the common items between the two lists.
  """

  common_items = []
  for item in list1:
    if item in list2:
      common_items.append(item)
  return common_items


# code in if block runs only from command line, code will NOT be executed if imported as a module
if __name__ == "__main__": 
  list1 = ["a", "b", "c", "d", "e"]
  list2 = ["b", "c", "d", "f", "g"]
  common_items = find_common_items(list1, list2)
  print(common_items)

['b', 'c', 'd']


In [14]:
# symbols are in past picks but also have bad data 
common_symbols = find_common_items(unique_symbols_picks, unique_symbols_bad_data)
print(common_symbols)

['AJRD', 'BKI', 'BTC-USD', 'CHT', 'CMG', 'CVLT', 'DVA', 'HY', 'KDNY', 'PKX', 'SRPT', 'WWE', 'XENE']


In [15]:
def subtract_items(list1, list2):
  """Subtracts items in list2 from items in list1.

  Args:
    list1: The first list.
    list2: The second list.

  Returns:
    A list of the items in list1 that are not in list2.
  """

  subtracted_items = []
  for item in list1:
    if item not in list2:
      subtracted_items.append(item)
  return subtracted_items

In [16]:
symbols_drop = subtract_items(unique_symbols_bad_data, common_symbols)  # don't drop symbols in past picks
symbols_drop .sort()
print(f'len(unique_symbols_bad_data): {len(unique_symbols_bad_data)}')
print(f'len(common_symbols): {len(common_symbols)}')
print(f'len(symbols_drop): {len(symbols_drop)}')

len(unique_symbols_bad_data): 296
len(common_symbols): 13
len(symbols_drop): 283


In [17]:
# symbols_drop = list(symbols_no_close) + list(symbols_no_volume) + list(symbols_same_volume) # combine symbols with no volume and no close
# print(f'combined symbols with no volume, same volume and no close, inculdes duplicate symbols: {len(symbols_drop)}')
# symbols_drop = list(set(symbols_drop))  # drop duplicate symbols
# symbols_drop .sort()


df_a = df.drop(symbols_drop, axis=1, level=0)  # drop symbols from OHLCA df
df_c = df_close.iloc[look_back_days::]
df_c = df_c.drop(symbols_drop, axis=1)
print(f'unique symbols dropped from df_a (adjOLHLV) and df_c (Close): {len(symbols_drop)}')

unique symbols dropped from df_a (adjOLHLV) and df_c (Close): 283


In [18]:
print(f'symbols with no volume:   {len(symbols_no_volume):>5,}')
print(f'symbols with same volume: {len(symbols_same_volume):>5,}')
print(f'symbols with no close:    {len(symbols_no_close):>5,}\n')
print(f'symbols total before drop:                                        {len(df_close.columns):>5,}')
print(f'unique symbols dropped from df OHLCVA (df_a) and df Close (df_c): {len(symbols_drop):>5,}\n')
print('                                          symbols     rows')
print(f'df adjOHLCV (df_a) after dropped symbols:   {len(df_a.columns)/5:>5,.0f}    {len(df_a):>5,}')
print(f'df Close (df_c) after dropped symbols:      {len(df_c.columns):>5,}    {len(df_c):>5,}')


symbols with no volume:      26
symbols with same volume:   232
symbols with no close:       62

symbols total before drop:                                        1,572
unique symbols dropped from df OHLCVA (df_a) and df Close (df_c):   283

                                          symbols     rows
df adjOHLCV (df_a) after dropped symbols:   1,289    1,500
df Close (df_c) after dropped symbols:      1,289    1,500


In [19]:
pickle_dump(df_a, path_data_dump, filename_pickled_df_a)
print(f'pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_a}')
pickle_dump(df_c, path_data_dump, filename_pickled_df_c)
print(f'pickled df Close after dropping symbols with no volume, same volume, and no close:\n{path_data_dump}{filename_pickled_df_c}')

pickled df adjOHLCV after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_OHLCV_clean
pickled df Close after dropping symbols with no volume, same volume, and no close:
C:/Users/ping/MyDrive/stocks/yfinance/VSCode_dump/df_close_clean


In [20]:
from myUtils import list_dump

f_symbols_df_close_clean = 'symbols_df_close_clean.csv'  # symbols text file
symbols_df_c = list(df_c)  # column names in df_c
list_dump(symbols_df_c, path_data_dump, f_symbols_df_close_clean)# df_c.columns.to_csv(f_symbols_df_close_clean)

In [21]:
df_a.tail()

A                                                    AA  \
                  Open        High         Low       Close     Volume   Open   
Date                                                                           
2023-09-19  112.370003  113.230003  111.720001  112.309998  2222400.0  28.83   
2023-09-20  113.080002  113.470001  112.169998  112.470001  1744900.0  28.65   
2023-09-21  112.029999  112.199997  109.500000  109.730003  1327900.0  28.68   
2023-09-22  109.730003  112.160004  109.110001  111.410004  2050500.0  28.27   
2023-09-25  110.760002  112.029999  110.529999  111.940002   987400.0  27.75   

                                                        ...       ZUMZ  \
                 High        Low      Close     Volume  ...       Open   
Date                                                    ...              
2023-09-19  28.989000  27.834999  28.340000  5151500.0  ...  16.510000   
2023-09-20  29.790001  28.549999  29.379999  8084100.0  ...  16.930000   
2023-09-21  28.680000  27.860001  27.870001  5487500.0  ...  17.430000   
2023-09-22  28.596001  28.049999  28.350000  5095000.0  ...  17.309999   
2023-09-25  27.770000  26.340000  26.629999  9298000.0  ...  17.299999   

                                                             ZWS             \
                 High        Low      Close    Volume       Open       High   
Date                                                                          
2023-09-19  17.120001  16.510000  16.950001  406200.0  28.120001  28.340000   
2023-09-20  17.719999  16.930000  17.670000  314600.0  28.260000  28.520000   
2023-09-21  17.590000  17.209999  17.270000  341000.0  27.760000  27.790001   
2023-09-22  17.530001  17.209999  17.410000  289500.0  27.600000  27.920000   
2023-09-25  17.860001  17.129999  17.809999  219600.0  27.490000  27.830000   

                                            
                  Low      Close    Volume  
Date                                        
2023-09-19  27.889999  28.139999  665000.0  
2023-09-20  27.959999  27.969999  576900.0  
2023-09-21  27.450001  27.480000  605500.0  
2023-09-22  27.490000  27.620001  675200.0  
2023-09-25  27.490000  27.740000  853400.0  

[5 rows x 6445 columns]

In [22]:
df_c.tail()

,A,AA,AAL,AAP,AAPL,AB,ABBV,ABC,ABM,ABR,...,YY,ZBH,ZBRA,ZD,ZG,ZION,ZTO,ZTS,ZUMZ,ZWS
Date,,,,,,,,,,,,,,,,,,,,,
2023-09-19,112.309998,28.340000,13.19,59.660000,179.070007,32.169998,153.490005,179.740005,40.680000,15.77,...,39.619999,121.610001,252.169998,66.889999,46.090000,35.889999,24.760000,181.979996,16.950001,28.139999
2023-09-20,112.470001,29.379999,13.06,60.119999,175.490005,32.000000,153.610001,182.779999,40.529999,15.48,...,39.700001,120.309998,235.830002,66.989998,46.750000,35.299999,24.719999,181.479996,17.670000,27.969999
2023-09-21,109.730003,27.870001,13.06,58.240002,173.929993,31.860001,153.429993,184.929993,40.090000,14.87,...,39.360001,117.800003,228.679993,67.629997,44.830002,34.160000,24.219999,176.630005,17.270000,27.480000
2023-09-22,111.410004,28.350000,12.90,57.599998,174.789993,31.320000,152.740005,185.949997,39.439999,14.89,...,40.049999,116.849998,227.149994,66.610001,44.160000,34.099998,24.459999,176.679993,17.410000,27.620001
2023-09-25,111.940002,26.629999,12.91,56.919998,176.080002,31.370001,154.649994,187.220001,39.439999,14.79,...,40.070000,117.349998,228.149994,64.989998,44.299999,34.580002,24.340000,177.720001,17.809999,27.740000
